## Homework 3

### CNN on CIFAR10

In [1]:
import numpy as np
from utils.misc import get_params_num, get_accuracy

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F

import torchvision
from torchvision import transforms
from IPython import display

torch.manual_seed(1)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Device: {}'.format(device))

Device: cpu


In [2]:
# import CIFAR10
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=2)

print("CIFAR images shape: {}".format(tuple(trainset[0][0].shape)))

Files already downloaded and verified
Files already downloaded and verified
CIFAR images shape: (3, 32, 32)


In [40]:
class Bottleneck(nn.Module):
    def __init__(self, in_channels, reduction_factor):
        super(Bottleneck, self).__init__()
        self.bottleneck = nn.Conv2d(in_channels,in_channels // reduction_factor, kernel_size=1)
        self.maxpo=nn.MaxPool2d(kernel_size=1)
        self.conv = nn.Conv2d(in_channels // reduction_factor, in_channels // reduction_factor, 
                              padding=1, kernel_size=3)
        self.maxpo2=nn.MaxPool2d(kernel_size=1)
        self.expansion = nn.Conv2d(in_channels // reduction_factor, in_channels, kernel_size=1)
        self.act = nn.ReLU()
        
    def forward(self, x):
        x = self.act(self.bottleneck(x))
        x = self.act(self.maxpo(x))
        x = self.act(self.conv(x))
        x = self.act(self.maxpo2(x))
        x = self.expansion(x)
        return x
    

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, reduction_factor):
        super(ResidualBlock, self).__init__()
        self.bottleneck = Bottleneck(in_channels, reduction_factor)
        
    def forward(self, x):
        return x + self.bottleneck(x)
    
    
class GlobalAveragePooling(nn.Module):
    def forward(self, x):
        return torch.mean(x.view(x.size(0), x.size(1), -1), dim=2)
    
    
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        C = 256
        n_classes = 10
        
        
        self.network = nn.Sequential(
                nn.Conv2d(in_channels=3, out_channels=C, kernel_size=3),
                nn.ELU(),
                nn.AvgPool2d(kernel_size=2),
                ResidualBlock(C, 2),
                nn.LeakyReLU(),
                GlobalAveragePooling(),
                nn.Linear(C, 100),
                nn.ReLU(),
                nn.Dropout(p=0.30),
                nn.Linear(100, n_classes)
        )
        
    def forward(self, x, verbose=False):
        return self.network(x)
        
        
net = CNN()
net.to(device)
print("# of parameters: {}".format(get_params_num(net)))
print(net)

# of parameters: 247382
CNN(
  (network): Sequential(
    (0): Conv2d(3, 256, kernel_size=(3, 3), stride=(1, 1))
    (1): ELU(alpha=1.0)
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (3): ResidualBlock(
      (bottleneck): Bottleneck(
        (bottleneck): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1))
        (maxpo): MaxPool2d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
        (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (maxpo2): MaxPool2d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
        (expansion): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1))
        (act): ReLU()
      )
    )
    (4): LeakyReLU(negative_slope=0.01)
    (5): GlobalAveragePooling()
    (6): Linear(in_features=256, out_features=100, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.3, inplace=False)
    (9): Linear(in_features=100, out_features=10, bias=True)
  )
)


In [41]:
lr = 0.001
epochs = 20

n_batches = len(trainloader)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=lr)

net.train() 
for e in range(epochs):
    for i, data in enumerate(trainloader):
        batch = data[0].to(device)
        labels = data[1].to(device)      
        outputs = net(batch)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if i % 50 == 0:
            print("[EPOCH]: {}, [BATCH]: {}/{}, [LOSS]: {}".format(e, i, n_batches, loss.item()))
            display.clear_output(wait=True)

[EPOCH]: 19, [BATCH]: 750/782, [LOSS]: 0.6723637580871582


In [42]:
acc_train = get_accuracy(trainloader, net, device=device)
acc_test = get_accuracy(testloader, net, device=device)
print("Train accuracy: {}\nTest accuracy: {}".format(acc_train, acc_test))

Train accuracy: 0.75874
Test accuracy: 0.7047


#### It can be considered a good performance
